# This notebook:
### plots beaching probability for the entire ocean with contours with temporal ENSO state contrast

In [1]:
# set working directory
import os
os.chdir('/dat1/openonic/Drifters') # directory

In [2]:
# dependencies
import pandas as pd
import scipy.interpolate as sci_int
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sci_stats
import cartopy.crs as ccrs
import geopandas as gpd

import matplotlib.colors as mcolors
plt.rcParams.update({'font.size': 18})

plt.rcParams['axes.titlesize'] = 24
plt.rcParams['axes.labelsize'] = 22
plt.rcParams['axes.labelsize'] = 22

In [3]:
# import undrogued ENSO data
beach = pd.read_csv('ENSO_beach.csv')
unbeach = pd.read_csv('ENSO_unbeach.csv')

In [4]:
# working from the end of 'ENSO_classification.ipynb'
# now, we consider the relative amount of time each drifter trajectory spends in the temporal el nino or la nina regimes. 
# If a drifter trajectory spends more than 80% of time in one regime or the other, we classify that drifter trajectory as an el nino or 
# la nina drifter trajectory, thus grouping our trajectories by temporal ENSO regime. If a drifter trajectory spends less than 80% of time 
# in either regime, then we classify that drifter trajectory as a 'mixed_ENSO' drifter trajectory

In [5]:
def classify_ENSO_trajectories(df, percent_parameter):
    # Group by id and ENSO state to calculate the percent in each group
    value_counts = df.groupby('id')['ENSO'].value_counts(normalize=True) * 100

    # initialize lists
    el_nino_traj = []
    la_nina_traj = []
    mixed_ENSO_traj = []

    # iterate through unique ids
    for ID in value_counts.index.get_level_values(0).unique():
        
        # get percent in el nino and la nina
        percent_el_nino = value_counts.get((ID, 'el nino'), 0)
        percent_la_nina = value_counts.get((ID, 'la nina'), 0)

        # classify id based on percentages
        if percent_el_nino >= percent_parameter:
            el_nino_traj.append(ID)
        elif percent_la_nina >= percent_parameter:
            la_nina_traj.append(ID)
        else:
            mixed_ENSO_traj.append(ID)

    return el_nino_traj, la_nina_traj, mixed_ENSO_traj

In [8]:
# 80 percent seems good
percent_parameter = 80

# beach
el_nino_traj_beach, la_nina_traj_beach, mixed_ENSO_traj_beach = classify_ENSO_trajectories(beach, percent_parameter)

# unbeach
el_nino_traj_unbeach, la_nina_traj_unbeach, mixed_ENSO_traj_unbeach = classify_ENSO_trajectories(unbeach, percent_parameter)

In [15]:
# Beach
# Separate data into nino, nina, mixed based on ID
beach_nino = beach[beach['id'].isin(el_nino_traj_beach)]
beach_nina = beach[beach['id'].isin(la_nina_traj_beach)]
beach_mixed = beach[beach['id'].isin(mixed_ENSO_traj_beach)]

# Unbeach
# Separate data into nino, nina, mixed based on ID
unbeach_nino = unbeach[unbeach['id'].isin(el_nino_traj_unbeach)]
unbeach_nina = unbeach[unbeach['id'].isin(la_nina_traj_unbeach)]
unbeach_mixed = unbeach[unbeach['id'].isin(mixed_ENSO_traj_unbeach)]